In [1]:
import xarray as xr
import xesmf as xe
import matplotlib.pyplot as plt
import glob
import os
import numpy as np
import datetime

/N/u/gongg/Quartz/anaconda3/envs/xesmf/lib/python3.10/site-packages/esmpy/interface/loadESMF.py:94: VersionWarning: ESMF installation version 8.8.0, ESMPy version 8.8.0b0
  warnings.warn("ESMF installation version {}, ESMPy version {}".format(


In [4]:
pattern = '/N/project/Zli_lab/gongg/stage4_data/stage4_nc_re/*.nc'
files = sorted(glob.glob(pattern))

ds_ref1 = xr.open_mfdataset("ref_04deg.nc")
ds_ref2 = xr.open_mfdataset("ref_.1deg.nc")

test_ds_wrf1 = xr.open_dataset('/N/project/Zli_lab/gongg/stage4_data/stage4_nc_re/2002060100.nc')
test_ds_wrf2 = xr.open_dataset('/N/project/Zli_lab/gongg/stage4_data/regrid_bl/20020601.nc')

regridder1 = xe.Regridder(test_ds_wrf1, ds_ref1, method="bilinear", periodic=False)
regridder2 = xe.Regridder(test_ds_wrf2, ds_ref2, method="conservative", periodic=False)

## do not rerun!

In [7]:
chunk_size = 24
for i in range(0, len(files), chunk_size): # 100 should be len(files_jja)
    chunk = files[i : i + chunk_size]
    
    ds_wrf = xr.open_mfdataset(chunk, combine='by_coords')

    ds_wrf_regrid = regridder2(regridder1(ds_wrf))
    
    yyyymmdd = os.path.basename(chunk[0])[0:8]                           

    out_file = f'/N/project/Zli_lab/gongg/stage4_data/regrid_cons/{yyyymmdd}.nc'
    
    ds_wrf_regrid.to_netcdf(out_file)

/tmp/ipykernel_3838633/1983577013.py:13: UserWarning: Times can't be serialized faithfully to int64 with requested units 'days since 2002-06-01'. Serializing with units 'hours since 2002-06-01' instead. Set encoding['dtype'] to floating point dtype to serialize with units 'days since 2002-06-01'. Set encoding['units'] to 'hours since 2002-06-01' to silence this warning .
  ds_wrf_regrid.to_netcdf(out_file)


### UTC to LST

In [ ]:
pattern_UTC  = '/N/project/Zli_lab/gongg/CONUS404_data/regrid_bl/*.nc'
files_UTC = sorted(glob.glob(pattern_UTC))

lon_ranges = [(-np.inf, -112.5), (-112.5, -97.5), (-97.5, -82.5), (-82.5, np.inf)]
utc_offsets = [-8, -7, -6, -5]

for file_path in files_UTC[4000:]:

    base_name = os.path.basename(file_path)  # e.g. '19800602.nc'
    date_part = base_name.replace('.nc', '') # e.g. '19800602'
    year      = date_part[0:4]               # '1980'
    month_str = date_part[4:6]               # '06'
    day_str   = date_part[6:8]               # '02'
    

    ds_sss = xr.open_dataset(file_path)
    original_times = ds_sss.Time.values  # 假设有 24 小时或更多
    
    for (lon_min, lon_max), offset in zip(lon_ranges, utc_offsets):

        mask = (ds_sss.lon >= lon_min) & (ds_sss.lon < lon_max)
        ds_lon_subset = ds_sss.where(mask, drop=True)
        

        if ds_lon_subset.lat.size > 0 and ds_lon_subset.lon.size > 0:

            adjusted_times = original_times + np.timedelta64(offset, 'h')
            ds_lon_subset = ds_lon_subset.assign_coords(time=adjusted_times)
            
            lat_min = ds_lon_subset.lat.min().values
            lat_max = ds_lon_subset.lat.max().values
            lat_splits = np.linspace(lat_min, lat_max, 10)  # 10个值分9段
            
            for i in range(len(lat_splits) - 1):
                lat_min_split = lat_splits[i]
                lat_max_split = lat_splits[i + 1]
                lat_mask = (ds_lon_subset.lat >= lat_min_split) & (ds_lon_subset.lat < lat_max_split)
                ds_lat_subset = ds_lon_subset.where(lat_mask, drop=True)
                ds_lat_subset = ds_lat_subset.drop_vars("Time")
                ds_lat_subset = ds_lat_subset.drop_vars("XTIME")
                output_folder = f'/N/project/Zli_lab/gongg/CONUS404_data/regrid_bl_LST/U{offset}{i}/'
                os.makedirs(output_folder, exist_ok=True)
                
                output_file = f"{year}-{month_str}-{day_str}.nc"
                output_path = os.path.join(output_folder, output_file)
                
                ds_lat_subset.to_netcdf(output_path)


In [2]:
ds = xr.open_dataset('/N/project/Zli_lab/gongg/stage4_data/stage4_nc_re/2024093023.nc')

In [3]:
ds

<xarray.Dataset> Size: 24MB
Dimensions:  (time: 1, y: 881, x: 1121)
Coordinates:
    lat      (y, x) float64 8MB ...
    lon      (y, x) float64 8MB ...
  * time     (time) datetime64[ns] 8B 2024-09-30T23:00:00
Dimensions without coordinates: y, x
Data variables:
    tp       (time, y, x) float64 8MB ...

In [2]:
ds = xr.open_dataset('/N/project/Zli_lab/gongg/stage4_data/stage4_daily/20240831.nc')